# This notebook is for EDA on May data

### Import needed modules

In [1]:
import pandas as pd

### Import .csv to DataFrame and check top and bottom of the data

In [2]:
may_df = pd.read_pickle('../data/may.pkl')
may_df.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
0,2019-05-01 00:01:41.247000,36.136822,-86.799877,PoweredLIRL1,93.0,Bird
1,2019-05-01 00:01:41.247000,36.191252,-86.772945,PoweredXWRWC,35.0,Bird
2,2019-05-01 00:01:41.247000,36.144752,-86.806293,PoweredMEJEH,90.0,Bird
3,2019-05-01 00:01:41.247000,36.162056,-86.774688,Powered1A7TC,88.0,Bird
4,2019-05-01 00:01:41.247000,36.150973,-86.783109,Powered2TYEF,98.0,Bird


In [3]:
may_df.tail()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
20292498,2019-05-31 23:59:42,36.1639,-86.7823,Poweredbb60ef04-c719-597f-805f-20978964fd8c,49.0,Jump
20292499,2019-05-31 23:59:42,36.1606,-86.7768,Powered9f5743ed-7f81-539b-9e37-6503d54445d3,68.0,Jump
20292500,2019-05-31 23:59:42,36.1606,-86.7769,Powered5744d1db-3bbe-55e5-9a23-a93828ba54fc,53.0,Jump
20292501,2019-05-31 23:59:42,36.1624,-86.7745,Poweredad1d6d3b-162a-5bec-bffd-cac4ac8dee1c,42.0,Jump
20292502,2019-05-31 23:59:42,36.1519,-86.7985,Powered8505008e-97c9-5003-8975-2ea6b2634fa0,76.0,Jump


### Check column attributes

In [4]:
may_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20283582 entries, 0 to 20292502
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   chargelevel  float64
 5   companyname  object 
dtypes: float64(3), object(3)
memory usage: 1.1+ GB


### Convert pubdatetime column from string to datetime format and create weekday column

In [5]:
may_df['pubdatetime'] = pd.to_datetime(may_df['pubdatetime'])
may_df['weekday'] = may_df['pubdatetime'].dt.weekday

### Check for null within columns

In [6]:
may_df.isnull().sum()

pubdatetime      0
latitude         0
longitude        0
sumdid           0
chargelevel    283
companyname      0
weekday          0
dtype: int64

### Drop nulls from chargelevel column

In [7]:
may_df.dropna(subset=['chargelevel'], inplace=True)
may_df.isnull().sum()

pubdatetime    0
latitude       0
longitude      0
sumdid         0
chargelevel    0
companyname    0
weekday        0
dtype: int64

In [8]:
may_df.companyname.value_counts()

Lime      7111655
Bird      4334006
Lyft      3112581
Jump      2984630
Spin      1300718
Gotcha     915314
Bolt       524395
Name: companyname, dtype: int64

### Check for dups within sumdid

In [9]:
may_df.sumdid.nunique()

8301

In [10]:
may_df.describe(include='O') # may not be useful

,sumdid,companyname
count,20283299,20283299
unique,8301,7
top,PoweredIE7H5RIN474PE,Lime
freq,8921,7111655


### Round latitude and longitude

In [11]:
may_df = may_df.round({'latitude': 3})

may_df = may_df.round({'longitude': 3})

### Create new dataframe and drop weekends rows

In [12]:
# Copy may_df dataframe into new dataframe for weekdays
may_wk_df = may_df.copy()

# get names of indexes for weekdays
index_names_wk = may_wk_df[may_wk_df['weekday'] >= 5].index

# drop weekend rows
may_wk_df.drop(index_names_wk, inplace = True)

#may_wknd_df.info()
may_wk_df.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,weekday
0,2019-05-01 00:01:41.247,36.137,-86.800,PoweredLIRL1,93.0,Bird,2
1,2019-05-01 00:01:41.247,36.191,-86.773,PoweredXWRWC,35.0,Bird,2
2,2019-05-01 00:01:41.247,36.145,-86.806,PoweredMEJEH,90.0,Bird,2
3,2019-05-01 00:01:41.247,36.162,-86.775,Powered1A7TC,88.0,Bird,2
4,2019-05-01 00:01:41.247,36.151,-86.783,Powered2TYEF,98.0,Bird,2


### Create new dataframe and drop weekday rows

In [13]:
# Copy may_df dataframe into new dataframe for weekends
may_wknd_df = may_df.copy()

# get names of indexes for weekends
index_names_wknd = may_wknd_df[may_wknd_df['weekday'] < 5].index

# drop weekday rows
may_wknd_df.drop(index_names_wknd, inplace = True)

#may_wknd_df.info()
may_wknd_df.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,weekday
1668376,2019-05-04 00:00:14,36.161,-86.775,Powered131810,70.0,Lyft,5
1668377,2019-05-04 00:00:17,36.163,-86.785,Powered757216,88.0,Lyft,5
1668378,2019-05-04 00:00:38,36.150,-86.806,Powered201452,16.0,Lyft,5
1668379,2019-05-04 00:00:44,36.144,-86.789,Powered809819,64.0,Lyft,5
1668380,2019-05-04 00:00:44,36.147,-86.799,Powered586079,100.0,Lyft,5


### Sort dataframes by sumdid and pubdatetime

In [14]:
may_wk_df = may_wk_df[['pubdatetime', 'sumdid', 'latitude', 'longitude', 'chargelevel', 'companyname', 'weekday']]
may_wk_df.sort_values(by=['sumdid','pubdatetime'], inplace=True)
may_wk_df.head()

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday
12757337,2019-05-24 00:04:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4
12761150,2019-05-24 00:09:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4
12765093,2019-05-24 00:14:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4
12769113,2019-05-24 00:19:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4
12773048,2019-05-24 00:24:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4


In [15]:
may_wknd_df = may_wknd_df[['pubdatetime', 'sumdid', 'latitude', 'longitude', 'chargelevel', 'companyname', 'weekday']]
may_wknd_df.sort_values(by=['sumdid','pubdatetime'], inplace=True)
may_wknd_df.head()

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday
13724123,2019-05-25 00:00:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.0,Bolt,5
13727674,2019-05-25 00:05:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.0,Bolt,5
13731219,2019-05-25 00:10:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.0,Bolt,5
13734513,2019-05-25 00:15:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.0,Bolt,5
13737950,2019-05-25 00:20:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.0,Bolt,5


### Calculate how long scooter is unused

In [16]:
#may_wknd_df['time_diff'] = may_wknd_df.groupby(['sumdid', 'latitude', 'longitude'])['pubdatetime'].diff().fillna(pd.Timedelta(0))
may_wknd_df['time_shift'] = may_wknd_df.groupby(['sumdid'])['pubdatetime'].shift(-1)
may_wknd_df['time_elaspe'] = may_wknd_df['time_shift'] - may_wknd_df['pubdatetime']
may_wknd_df['time_diff'] = may_wknd_df.groupby(['latitude', 'longitude'])['pubdatetime'].diff().fillna(pd.Timedelta(0))
may_wknd_df.head(25)

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday,time_shift,time_elaspe,time_diff
13724123,2019-05-25 00:00:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:05:36,0 days 00:05:00,0 days 00:00:00
13727674,2019-05-25 00:05:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:10:37,0 days 00:05:01,0 days 00:05:00
13731219,2019-05-25 00:10:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:15:37,0 days 00:05:00,0 days 00:05:01
13734513,2019-05-25 00:15:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:20:37,0 days 00:05:00,0 days 00:05:00
13737950,2019-05-25 00:20:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 02:10:41,0 days 01:50:04,0 days 00:05:00
13815770,2019-05-25 02:10:41,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,51.24,Bolt,5,2019-05-25 02:15:43,0 days 00:05:02,0 days 00:00:00
13819049,2019-05-25 02:15:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,52.38,Bolt,5,2019-05-25 02:20:44,0 days 00:05:01,0 days 00:05:02
13822282,2019-05-25 02:20:44,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,52.95,Bolt,5,2019-05-25 02:25:43,0 days 00:04:59,0 days 00:05:01
13825650,2019-05-25 02:25:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,53.23,Bolt,5,2019-05-25 02:30:43,0 days 00:05:00,0 days 00:04:59
13828841,2019-05-25 02:30:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,53.51,Bolt,5,2019-05-25 02:35:44,0 days 00:05:01,0 days 00:05:00


In [17]:
#may_wk_df['time_elaspe'] = may_wk_df.groupby(['sumdid', 'latitude', 'longitude'])['pubdatetime'].diff().fillna(pd.Timedelta(0))
may_wk_df['time_shift'] = may_wk_df.groupby(['sumdid'])['pubdatetime'].shift(-1)
may_wk_df['time_elaspe'] = may_wk_df['time_shift'] - may_wk_df['pubdatetime']
may_wk_df.head()

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday,time_shift,time_elaspe
12757337,2019-05-24 00:04:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:09:42,0 days 00:05:00
12761150,2019-05-24 00:09:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:14:42,0 days 00:05:00
12765093,2019-05-24 00:14:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:19:42,0 days 00:05:00
12769113,2019-05-24 00:19:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:24:43,0 days 00:05:01
12773048,2019-05-24 00:24:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:29:43,0 days 00:05:00


### Locate when the scooter is taken on a trip
- Weekend

In [18]:
may_wknd_df['prev_latitude'] = may_wknd_df.groupby(['sumdid'])['latitude'].shift(1)
may_wknd_df['prev_longitude'] = may_wknd_df.groupby(['sumdid'])['longitude'].shift(1)
may_wknd_df['lat_dff'] = may_wknd_df['prev_latitude'] - may_wknd_df['latitude']
may_wknd_df['long_dff'] = may_wknd_df['prev_longitude'] - may_wknd_df['longitude']
may_wknd_df.head(50)

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday,time_shift,time_elaspe,time_diff,prev_latitude,prev_longitude,lat_dff,long_dff
13724123,2019-05-25 00:00:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:05:36,0 days 00:05:00,0 days 00:00:00,NaN,NaN,NaN,NaN
13727674,2019-05-25 00:05:36,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:10:37,0 days 00:05:01,0 days 00:05:00,36.154,-86.785,0.000,0.000
13731219,2019-05-25 00:10:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:15:37,0 days 00:05:00,0 days 00:05:01,36.154,-86.785,0.000,0.000
13734513,2019-05-25 00:15:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 00:20:37,0 days 00:05:00,0 days 00:05:00,36.154,-86.785,0.000,0.000
13737950,2019-05-25 00:20:37,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154,-86.785,100.00,Bolt,5,2019-05-25 02:10:41,0 days 01:50:04,0 days 00:05:00,36.154,-86.785,0.000,0.000
13815770,2019-05-25 02:10:41,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,51.24,Bolt,5,2019-05-25 02:15:43,0 days 00:05:02,0 days 00:00:00,36.154,-86.785,0.026,-0.006
13819049,2019-05-25 02:15:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,52.38,Bolt,5,2019-05-25 02:20:44,0 days 00:05:01,0 days 00:05:02,36.128,-86.779,0.000,0.000
13822282,2019-05-25 02:20:44,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,52.95,Bolt,5,2019-05-25 02:25:43,0 days 00:04:59,0 days 00:05:01,36.128,-86.779,0.000,0.000
13825650,2019-05-25 02:25:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,53.23,Bolt,5,2019-05-25 02:30:43,0 days 00:05:00,0 days 00:04:59,36.128,-86.779,0.000,0.000
13828841,2019-05-25 02:30:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.128,-86.779,53.51,Bolt,5,2019-05-25 02:35:44,0 days 00:05:01,0 days 00:05:00,36.128,-86.779,0.000,0.000


- Weekday

In [19]:
may_wk_df['prev_latitude'] = may_wk_df.groupby(['sumdid'])['latitude'].shift(1)
may_wk_df['prev_longitude'] = may_wk_df.groupby(['sumdid'])['longitude'].shift(1)
may_wk_df['lat_dff'] = may_wk_df['prev_latitude'] - may_wk_df['latitude']
may_wk_df['long_dff'] = may_wk_df['prev_longitude'] - may_wk_df['longitude']
may_wk_df.head()

,pubdatetime,sumdid,latitude,longitude,chargelevel,companyname,weekday,time_shift,time_elaspe,prev_latitude,prev_longitude,lat_dff,long_dff
12757337,2019-05-24 00:04:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:09:42,0 days 00:05:00,NaN,NaN,NaN,NaN
12761150,2019-05-24 00:09:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:14:42,0 days 00:05:00,36.162,-86.775,0.0,0.0
12765093,2019-05-24 00:14:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:19:42,0 days 00:05:00,36.162,-86.775,0.0,0.0
12769113,2019-05-24 00:19:42,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:24:43,0 days 00:05:01,36.162,-86.775,0.0,0.0
12773048,2019-05-24 00:24:43,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.162,-86.775,100.0,Bolt,4,2019-05-24 00:29:43,0 days 00:05:00,36.162,-86.775,0.0,0.0


In [20]:
#may_wk_df['time_elaspe'].value_counts()